### Importing Necessary Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import datetime

### Specify path for exporting the results to a csv file

In [2]:
export_path = "/Users/alay/output.csv" #Don't forget to add '.csv' at the end of the path

### Helper functions to process and parse the data rows which will be scraped from the website

#### This function takes in row data as argument, and processes one row at a time. For each row, it verifies if any of its launch was successful or not, and increments the orbital launch value in the result dataframe whenever it finds a successful launch

In [3]:
def parseLaunches(start_row_index, rows, df):
    i = start_row_index
    while i<len(rows):
        if(len(rows[i]) == len(columns_rocket*2)):
            date = getLaunchDate(rows[i])
            rowspan = int(rows[i].find('td')['rowspan'])
            index = 1
            while(index < rowspan and i < len(rows)):
                i+=1
                if(len(rows[i]) == len(columns_payload)*2):
                    outcome = rows[i].find_all('td')[5].text
                    outcome = outcome.strip().lower()
                    if(outcome in success):
                        df.loc[date] +=1
                        break
                index+=1
            i+=1
        else:
            i+=1

#### This is a helper function used in the above mentioned (parseLaunches()) function which takes in a row having date as argument, and returns the date in ISO 8601 format

In [4]:
def getLaunchDate(row):
    date = row.find('span').text
    date = re.sub('[^0-9a-zA-Z]+', ' ', date)
    date = date.split()[:2]
    date = date[0] + " " + date[1] + " 2019"
    date = datetime.strptime(date, '%d %B %Y')
    date = datetime.strftime(date, '%Y-%m-%dT%H:%M:%S+%H:%M')
    return(date)

In [5]:
success = ["successful", "operational", "en route"] ### Storing the success messages in a success List

### Creating an empty data frame having dates in requires ISO 8601 format as indices and values as attributes. The initial values for all rows will be set to 0.

In [6]:
start_date = datetime.strptime('1 January 2019', '%d %B %Y')
end_date = datetime.strptime('31 December 2019', '%d %B %Y')
index = pd.date_range(start_date, end_date, freq='D')
index

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08',
               '2019-01-09', '2019-01-10',
               ...
               '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
               '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

In [7]:
df = pd.DataFrame(index = index, columns = ['value'])
df.index.names = ['date']
df['value'] = 0
df.index = df.index.map(lambda x: datetime.strftime(x, '%Y-%m-%dT%H:%M:%S+%H:%M'))
df

,value
date,
2019-01-01T00:00:00+00:00,0
2019-01-02T00:00:00+00:00,0
2019-01-03T00:00:00+00:00,0
2019-01-04T00:00:00+00:00,0
2019-01-05T00:00:00+00:00,0
...,...
2019-12-27T00:00:00+00:00,0
2019-12-28T00:00:00+00:00,0
2019-12-29T00:00:00+00:00,0


### Scraping Data From Source Website

The general procedure to get data from websites is:

1. Use requests to connect to a URL and get data from it
2. Create a BeautifulSoup object
3. Get attributes of the BeautifulSoup object (i.e. the HTML elements that we want)

In [8]:
# getting HTML from Wikipedia Source Web Page
url = "https://en.wikipedia.org/wiki/2019_in_spaceflight#Orbital_launches"
response = requests.get(url)

# create a BeautifulSoup object
soup = BeautifulSoup(response.text, "html.parser")
    
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   2019 in spaceflight - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XhozxQpAADkAAA07RlYAAABO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"2019_in_spaceflight","wgTitle":"2019 in spaceflight","wgCurRevisionId":934594174,"wgRevisionId":934594174,"wgArticleId":34130425,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 Chinese-language sources (zh)","CS1 Spanish

### Grabbing the Orbital Table which is of our primary interest

In [9]:
table = soup.find('table', {'class':'wikitable'})
print(table.prettify())

<table border="1" class="wikitable collapsible" width="100%">
 <tbody>
  <tr>
   <th rowspan="3" scope="col">
    Date and time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/Rocket" title="Rocket">
     Rocket
    </a>
   </th>
   <th scope="col">
    Flight number
   </th>
   <th colspan="2" scope="col">
    <a class="mw-redirect" href="/wiki/Launch_site" title="Launch site">
     Launch site
    </a>
   </th>
   <th colspan="2" scope="col">
    <a href="/wiki/Launch_service_provider" title="Launch service provider">
     LSP
    </a>
   </th>
  </tr>
  <tr>
   <td style="text-align:center;background-color:#e3e9e9;">
    <b>
     <a href="/wiki/Payload" title="Payload">
      Payload
     </a>
    </b>
    <br/>
    (⚀ =
    <a href="/wiki/CubeSat" title="CubeSat">
     CubeSat
    </a>
    )
   </td>
   <td style="text-align:center;background-color:#e3e9e9;">
    <b>
     Op

In [10]:
rows = table.find_all('tr')

columns_rocket =  [v.text.replace('\n', '') for v in rows[0].find_all('th')]
columns_payload = [v.text.replace('\n', '') for v in rows[1].find_all('td')]

print("Column Names for Rocket:")
print(columns_rocket)

print("")

print("Column Names for Payload:")
print(columns_payload)

print("")

print("Total number of entries in Table:")
print(len(rows))

Column Names for Rocket:
['Date and time (UTC)', 'Rocket', 'Flight number', 'Launch site', 'LSP']

Column Names for Payload:
['Payload  (⚀ = CubeSat)', 'Operator', 'Orbit', 'Function', 'Decay (UTC)', 'Outcome']

Total number of entries in Table:
425


### Calling above defined helper functions to process rows and populate data frame

In [11]:
# Since first two rows are header rows, hence we will start processing from 3rd row
start_row_index=3
parseLaunches(start_row_index, rows, df)

### Resultant DataFrame

In [12]:
df.head()

,value
date,
2019-01-01T00:00:00+00:00,0
2019-01-02T00:00:00+00:00,0
2019-01-03T00:00:00+00:00,0
2019-01-04T00:00:00+00:00,0
2019-01-05T00:00:00+00:00,0


In [13]:
df[df['value'] > 0]

,value
date,
2019-01-10T00:00:00+00:00,1
2019-01-11T00:00:00+00:00,1
2019-01-18T00:00:00+00:00,1
2019-01-19T00:00:00+00:00,1
2019-01-21T00:00:00+00:00,1
...,...
2019-12-18T00:00:00+00:00,1
2019-12-20T00:00:00+00:00,1
2019-12-24T00:00:00+00:00,1


In [14]:
export_csv = df.to_csv(export_path, header=True)